In [ ]:
import os
import random
import sys
import warnings
import numpy as np
import pandas as pd
from itertools import chain
from skimage.io import imread, imsave, imshow, imread_collection, concatenate_images
from skimage.color import gray2rgb
from skimage.transform import resize
from skimage.morphology import label
from keras.utils import Progbar

from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.core import Dropout, Lambda, Activation
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam, Adamax, Nadam, RMSprop
from keras.layers.normalization import BatchNormalization
import time
import cv2

from dataset_stage2 import DSB18Dataset, _DEFAULT_DS_NUCLEI_VAL_TEST_OPTIONS, _DEFAULT_DS_CONTOURS_VAL_TEST_OPTIONS
from post_stage2 import Post, _DEFAULT_PROC_OPTIONS
from visualize import display_sem_label_gt_vs_pred, display_sem_label_gt_and_pred
from visualize import display_image_and_pred_sem_label, display_image_and_pred_masks
from submission_stage2 import DSB18Submission

# Import cyclic LR
from clr_callback import *

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
seed = 42
random.seed = seed
# np.random.seed = seed
smooth = 1.
#epochs = 48
#epochs = 50
#epochs = 70
#epochs = 150
epochs = 300

# Number of test samples to display in notebook
num_samples = 100

In [ ]:
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
# Note: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    dataset_root = "E:/datasets/dsb18.retrain"
else:
    dataset_root = "/media/EDrive/datasets/dsb18.retrain"
TRAIN_PATH = dataset_root + "/stage1_train/"
TEST_PATH = dataset_root + "/stage2_test_final/"

In [ ]:
import glob

train_folder = TRAIN_PATH
samples = glob.glob(train_folder+'/**/')
n_samples = len(samples)
print ('number of samples:',n_samples)
print(samples[0])

In [ ]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
# Function read train images and mask return as nump array
def read_train_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()

    if os.path.isfile("train_img_256x256_retrained.npy"):
        print("Train file train_img_256x256_retrained.npy loaded from memory")
        X_train = np.load("train_img_256x256_retrained.npy")
    else:
        a = Progbar(len(train_ids))
        for n, id_ in enumerate(train_ids):
            path = TRAIN_PATH + id_
            img = imread(path + '/images/' + id_ + '.png')
            if len(img.shape) == 2:
                img = gray2rgb(img)
                if img.dtype == np.uint16:
                    img = (img // 255).astype(np.uint8)
            else:
                img = img[:,:,:IMG_CHANNELS]
            img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
            X_train[n] = img
            a.update(n)
        np.save("train_img_256x256_retrained.npy",X_train)
        
    if os.path.isfile("train_mask_256x256_retrained.npy"):
        print("Train file train_mask_256x256_retrained.npy loaded from memory")
        Y_train = np.load("train_mask_256x256_retrained.npy")
    else:
        a = Progbar(len(train_ids))
        for n, id_ in enumerate(train_ids):
            path = TRAIN_PATH + id_
            mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
            for mask_file in next(os.walk(path + '/masks/'))[2]:
                mask_ = imread(path + '/masks/' + mask_file)
                mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                            preserve_range=True), axis=-1)
                mask = np.maximum(mask, mask_)
            Y_train[n] = mask
            a.update(n)
        np.save("train_mask_256x256_retrained.npy",Y_train)
        
    return X_train,Y_train

In [ ]:
# Function read train images and contours return as nump array
def read_train_contour_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()

    if os.path.isfile("train_img_256x256_retrained.npy"):
        print("Train file train_img_256x256_retrained.npy loaded from memory")
        X_train = np.load("train_img_256x256_retrained.npy")
    else:
        a = Progbar(len(train_ids))
        for n, id_ in enumerate(train_ids):
            path = TRAIN_PATH + id_
            img = imread(path + '/images/' + id_ + '.png')
            if len(img.shape) == 2:
                img = gray2rgb(img)
                if img.dtype == np.uint16:
                    img = (img // 255).astype(np.uint8)
            else:
                img = img[:,:,:IMG_CHANNELS]
            img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
            X_train[n] = img
            a.update(n)
        np.save("train_img_256x256_retrained.npy",X_train)
        
    if os.path.isfile("train_2px_contours_256x256_retrained.npy"):
        print("Train file train_2px_contours_256x256_retrained.npy loaded from memory")
        Y_train = np.load("train_2px_contours_256x256_retrained.npy")
    else:
        a = Progbar(len(train_ids))
        for n, id_ in enumerate(train_ids):
            path = TRAIN_PATH + id_
            mask = imread(path + '/contours_2px/' + id_ + '.png')
            mask = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
            mask = np.expand_dims(mask, axis=-1)
            Y_train[n] = mask.astype(np.bool)
            a.update(n)
        np.save("train_2px_contours_256x256_retrained.npy",Y_train)
        
    return X_train,Y_train

In [ ]:
# Function to read test images and return as numpy array
def read_test_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    sizes_test = []
    print('\nGetting and resizing test images ... ')
    sys.stdout.flush()
    if os.path.isfile("test_img_256x256_retrained.npy") and os.path.isfile("test_size_256x256_retrained.npy"):
        print("Test file loaded from memory")
        X_test = np.load("test_img_256x256_retrained.npy")
        sizes_test = np.load("test_size_256x256_retrained.npy")
        return X_test,sizes_test
    b = Progbar(len(test_ids))
    for n, id_ in enumerate(test_ids):
        path = TEST_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')
        if len(img.shape) == 2:
            img = gray2rgb(img)
            if img.dtype == np.uint16:
                img = (img // 255).astype(np.uint8)
        else:
            img = img[:,:,:IMG_CHANNELS]
        sizes_test.append([img.shape[0], img.shape[1]])
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_test[n] = img
        b.update(n)
    np.save("test_img_256x256_retrained.npy",X_test)
    np.save("test_size_256x256_retrained.npy",sizes_test)
    return X_test,sizes_test

In [ ]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [ ]:
# Iterate over the test IDs and generate run-length encodings for each seperate mask identified by skimage
def mask_to_rle(preds_test_upsampled):
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ids):
        rle = list(prob_to_rles(preds_test_upsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
    return new_test_ids,rles

In [ ]:
# Metric function
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Loss funtion
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
# Number of image channels (for example 3 in case of RGB, or 1 for grayscale images)
INPUT_CHANNELS = 3
# Number of output masks (1 in case you predict only one type of objects)
OUTPUT_MASK_CHANNELS = 1

def double_conv_layer(x, size, dropout, batch_norm):
    axis = 3
    conv = Conv2D(size, (3, 3), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    if dropout > 0:
        conv = Dropout(dropout)(conv)
    return conv

def get_unet(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3, dropout_val=0.0, batch_norm=True):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)

    filters = 32
    axis = 3

    conv_224 = double_conv_layer(inputs, filters, dropout_val, batch_norm)
    pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)

    conv_112 = double_conv_layer(pool_112, 2*filters, dropout_val, batch_norm)
    pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)

    conv_56 = double_conv_layer(pool_56, 4*filters, dropout_val, batch_norm)
    pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)

    conv_28 = double_conv_layer(pool_28, 8*filters, dropout_val, batch_norm)
    pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)

    conv_14 = double_conv_layer(pool_14, 16*filters, dropout_val, batch_norm)
    pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)

    conv_7 = double_conv_layer(pool_7, 32*filters, dropout_val, batch_norm)

    up_14 = concatenate([UpSampling2D(size=(2, 2))(conv_7), conv_14], axis=axis)
    up_conv_14 = double_conv_layer(up_14, 16*filters, dropout_val, batch_norm)

    up_28 = concatenate([UpSampling2D(size=(2, 2))(up_conv_14), conv_28], axis=axis)
    up_conv_28 = double_conv_layer(up_28, 8*filters, dropout_val, batch_norm)

    up_56 = concatenate([UpSampling2D(size=(2, 2))(up_conv_28), conv_56], axis=axis)
    up_conv_56 = double_conv_layer(up_56, 4*filters, dropout_val, batch_norm)

    up_112 = concatenate([UpSampling2D(size=(2, 2))(up_conv_56), conv_112], axis=axis)
    up_conv_112 = double_conv_layer(up_112, 2*filters, dropout_val, batch_norm)

    up_224 = concatenate([UpSampling2D(size=(2, 2))(up_conv_112), conv_224], axis=axis)
    up_conv_224 = double_conv_layer(up_224, filters, 0, batch_norm)

    conv_final = Conv2D(OUTPUT_MASK_CHANNELS, (1, 1))(up_conv_224)
    conv_final = BatchNormalization(axis=axis)(conv_final)
    conv_final = Activation('sigmoid')(conv_final)

   # model = Model(inputs, conv_final, name="YTS_UNET_224")

   # outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[conv_final], name="YTS_UNET_224")
    #myoptim = Adam(lr=0.0009, decay=0.0)
    #myoptim = Adam(lr=0.001, decay=0.0)
    #myoptim = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    #myoptim = Adamax(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001)
    #myoptim = Adamax(lr=0.0018, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    myoptim = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, clipnorm=5.0)
    model.compile(optimizer=myoptim,loss='binary_crossentropy', metrics=[dice_coef])
    return model

## Train nuclei semantic segmentation

In [ ]:
# get train_data
train_img,train_mask = read_train_data()

# get test_data
test_img,test_img_sizes = read_test_data()

In [ ]:
# get u_net model
u_net = get_unet()

In [ ]:
t0 = time.time()

# fit model on train_data
print("\nTraining...")
#u_net.fit(train_img,train_mask,batch_size=16,epochs=epochs)
# Fit model
clr_triangular = CyclicLR(mode='exp_range', gamma=0.99994) # Add cyclic LR
earlystopper = EarlyStopping(patience=20, verbose=1)
# Reload for cyclic LR finetuning
# u_net.load_weights("model_train_unet_512x512_nuclei.h5")
checkpointer = ModelCheckpoint('model_retrained_unet_256x256_nuclei_cyclic_lr.h5', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint('model_train_unet_256x256_nuclei.h5', verbose=1, save_best_only=True)
results = u_net.fit(train_img, train_mask, validation_split=0.1, batch_size=8, epochs=epochs, 
                    callbacks=[clr_triangular, earlystopper, checkpointer])

t1 = time.time()                                                                              
total = t1-t0
print('Training the model took ', total/60, 'minutes')

## Run nuclei semantic segmentation on test split

In [ ]:
print("Predicting")
# Predict on test data
# test_mask = u_net.predict(test_img,verbose=1)
model = load_model('model_retrained_unet_256x256_nuclei_cyclic_lr.h5', custom_objects={'dice_coef': dice_coef})
# model = load_model('model_train_unet_256x256_nuclei.h5', custom_objects={'dice_coef': dice_coef})
test_mask = model.predict(test_img, batch_size=1, verbose=1)

In [ ]:
# Create list of upsampled test masks
test_mask_upsampled = []
for i in range(len(test_mask)):
    test_mask_upsampled.append(resize(np.squeeze(test_mask[i]),
                                       (test_img_sizes[i][0],test_img_sizes[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
# Save predicted semantic segmentation to disk
b = Progbar(len(test_ids))
for n, id_ in enumerate(test_ids):
    folder = TEST_PATH + id_ + '/pred_sem_label_raw/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    output_file = folder + id_ + '.png'
    if os.path.exists(output_file):
        os.remove(output_file)
    imsave(output_file, test_mask_upsampled[n])
    folder = TEST_PATH + id_ + '/pred_sem_label/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    output_file = folder + id_ + '.png'
    if os.path.exists(output_file):
        os.remove(output_file)
    test_mask_thresholded = np.where(test_mask_upsampled[n] > 0.5, 255, 0)
    imsave(output_file, test_mask_thresholded)
    b.update(n)

## Train contours semantic segmentation

In [ ]:
# get train_data
train_img,train_contours = read_train_contour_data()

# get test_data
test_img,test_img_sizes = read_test_data()

In [ ]:
# get u_net model
u_net = get_unet()

In [ ]:
t0 = time.time()

# fit model on train_data
print("\nTraining...")
#u_net.fit(train_img,train_mask,batch_size=16,epochs=epochs)
# Fit model
earlystopper = EarlyStopping(patience=20, verbose=1)
# u_net.load_weights("model_train_unet_256x256_nuclei.h5")
# Reload for cyclic LR finetuning
clr_triangular = CyclicLR(mode='exp_range', gamma=0.99994) # Add cyclic LR
# u_net.load_weights("model_train_unet_256x256_contours_2px.h5")
checkpointer = ModelCheckpoint('model_retrained_unet_256x256_contours_2px_cyclic_lr.h5', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint('model_train_unet_256x256_contours.h5', verbose=1, save_best_only=True)
results = u_net.fit(train_img, train_contours, validation_split=0.1, batch_size=8, epochs=epochs, 
                    callbacks=[clr_triangular, earlystopper, checkpointer])

t1 = time.time()                                                                              
total = t1-t0
print('Training the model took ', total/60, 'minutes')

## Run contours semantic segmentation on test split

In [ ]:
print("Predicting")
# Predict on test data
#test_mask = u_net.predict(test_img,verbose=1)
model = load_model('model_retrained_unet_256x256_contours_2px_cyclic_lr.h5', custom_objects={'dice_coef': dice_coef})
test_contours = model.predict(test_img, batch_size=1, verbose=1)

In [ ]:
# Create list of upsampled test contours
test_contours_upsampled = []
for i in range(len(test_contours)):
    test_contours_upsampled.append(resize(np.squeeze(test_contours[i]),
                                       (test_img_sizes[i][0],test_img_sizes[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
# Save predicted contours to disk
b = Progbar(len(test_ids))
for n, id_ in enumerate(test_ids):
    folder = TEST_PATH + id_ + '/pred_contours_2px_raw/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    output_file = folder + id_ + '.png'
    if os.path.exists(output_file):
        os.remove(output_file)
    imsave(output_file, test_contours_upsampled[n])
    folder = TEST_PATH + id_ + '/pred_contours_2px/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    output_file = folder + id_ + '.png'
    if os.path.exists(output_file):
        os.remove(output_file)
    test_contour_thresholded = np.where(test_contours_upsampled[n] > 0.5, 255, 0)
    imsave(output_file, test_contour_thresholded)
    b.update(n)

## Post-process segmented nuclei and contours to generate instance segmentations
> Had to split `test.txt` in two (`test_part1.txt` and `test_part2.txt`) and run post-processing in two steps because of OOM errors.

### Part 1 (1510 files)

In [ ]:
# Instantiate default post-processor
post = Post(ds_root=dataset_root, options=_DEFAULT_PROC_OPTIONS)
post.print_config()

In [ ]:
# Run the post-processor
post.process()

### Part 2 (1509 files)

In [ ]:
# Instantiate default post-processor
post = Post(ds_root=dataset_root, options=_DEFAULT_PROC_OPTIONS)
post.print_config()

In [ ]:
# Run the post-processor
post.process()

## Visualize final results (first few)

In [ ]:
# Load the test dataset test
options=_DEFAULT_DS_CONTOURS_VAL_TEST_OPTIONS.copy()
options['mode'] = 'instance_masks'
options['input_channels'] = 3
ds = DSB18Dataset(phase='test', ds_root=dataset_root, options=options)
ds.print_config()

In [ ]:
images, pred_inst_masks, _, IDs = ds.get_rand_samples_with_inst_masks(num_samples, 'test_with_preds', return_IDs=True, deterministic=True)
for image, pred_mask, ID in zip(images, pred_inst_masks, IDs):
    # Display two images: original RGB image and RGB image with predicted instance masks overlayed.
    # Each mask is displayed in a unique color.
    # This is useful to debug issues with instance separation.
    display_image_and_pred_masks(image, pred_mask, ID)

## Generate submission

In [ ]:
# Create a Submission object
submit = DSB18Submission(ds_root=dataset_root)
submit.print_config()

In [ ]:
# RLE encode predicted instance masks
submit.encode_predicted_instance_masks()

In [ ]:
# Create submission DataFrame and generate submission CSV
submit.to_csv()
print(submit.instance_masks_csv)